# 🚀 Atheria 4: Entrenamiento Progresivo (Long-Running)

Notebook optimizado para **entrenamientos largos** (muchas horas) con aprovechamiento máximo de cuota de GPU.

## ✨ Características

- 🔄 **Auto-guardado en Google Drive** - Sincronización configurable
- 📊 **Monitoreo de recursos** - GPU%, RAM, tiempo de sesión
- ⚡ **Auto-recuperación** - Continúa automáticamente desde checkpoints
- ⏰ **Límite de tiempo automático** - Guardado de emergencia antes de timeout
- 📈 **Visualización en tiempo real** - Progreso, métricas, recursos
- 💾 **Smart Checkpointing** - Solo guarda mejores modelos + último

## 📋 Cuotas de GPU

- **Colab Free**: ~12 horas/día (variable)
- **Colab Pro**: ~24 horas continuas
- **Kaggle**: 30 horas/semana (T4/P100)

## 🎯 Workflow Recomendado

1. Configurar experimento (Sección 4)
2. Ejecutar todas las celdas automáticamente
3. Dejar corriendo sin supervisión
4. Checkpoints se guardan automáticamente en Drive
5. Si se desconecta: ejecutar de nuevo, auto-recupera desde Drive

---
## 📦 Sección 1: Setup y Detección de Entorno

In [ ]:
# Detectar entorno (Kaggle o Colab)
import os
import sys
from pathlib import Path

# IMPORTANTE: Verificar Kaggle PRIMERO (tiene google.colab instalado pero no funciona)
IN_KAGGLE = os.path.exists("/kaggle/input") or os.path.exists("/kaggle/working")

# Solo si NO es Kaggle, verificar Colab
if not IN_KAGGLE:
    try:
        import google.colab
        # Verificar que realmente estemos en Colab (existe /content)
        IN_COLAB = os.path.exists("/content")
    except:
        IN_COLAB = False
else:
    IN_COLAB = False

ENV_NAME = "Kaggle" if IN_KAGGLE else "Colab" if IN_COLAB else "Local"
print(f"🌍 Entorno detectado: {ENV_NAME}")

# Instalar dependencias básicas
print("📦 Instalando dependencias...")
%pip install -q snntorch scikit-learn matplotlib

# Para Colab: instalar pybind11 (opcional, solo para motor nativo)
if IN_COLAB:
    %pip install -q pybind11

print("✅ Dependencias instaladas")

---
## 💾 Sección 2: Google Drive - Montaje y Configuración

In [ ]:
# Montar Google Drive (solo Colab)
if IN_COLAB:
    from google.colab import drive
    print("📁 Montando Google Drive...")
    drive.mount('/content/drive')
    DRIVE_ROOT = Path("/content/drive/MyDrive/Atheria")
    print(f"✅ Drive montado en: {DRIVE_ROOT}")
elif IN_KAGGLE:
    # En Kaggle, usar /kaggle/working como alternativa
    DRIVE_ROOT = Path("/kaggle/working/atheria_checkpoints")
    print(f"📁 Usando directorio local: {DRIVE_ROOT}")
else:
    DRIVE_ROOT = Path.home() / "atheria_checkpoints"
    print(f"💻 Usando directorio local: {DRIVE_ROOT}")

# Crear estructura de carpetas
DRIVE_CHECKPOINT_DIR = DRIVE_ROOT / "checkpoints"
DRIVE_LOGS_DIR = DRIVE_ROOT / "logs"
DRIVE_EXPORTS_DIR = DRIVE_ROOT / "exports"

for directory in [DRIVE_CHECKPOINT_DIR, DRIVE_LOGS_DIR, DRIVE_EXPORTS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

print(f"\n📂 Estructura de carpetas creada:")
print(f"  - Checkpoints: {DRIVE_CHECKPOINT_DIR}")
print(f"  - Logs: {DRIVE_LOGS_DIR}")
print(f"  - Exports: {DRIVE_EXPORTS_DIR}")

---
## 🔧 Sección 3: Clonar Proyecto Atheria

In [ ]:
# Configurar ruta del proyecto
if IN_KAGGLE:
    PROJECT_ROOT = Path("/kaggle/working/Atheria")
    if not PROJECT_ROOT.exists():
        print("⚠️ Proyecto no encontrado. Clonando desde GitHub...")
        !git clone https://github.com/Jonakss/Atheria.git /kaggle/working/Atheria
elif IN_COLAB:
    PROJECT_ROOT = Path("/content/Atheria")
    if not PROJECT_ROOT.exists():
        print("📥 Clonando proyecto desde GitHub...")
        !git clone https://github.com/Jonakss/Atheria.git /content/Atheria
else:
    # Local: asumir que estamos en notebooks/
    PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()

# Agregar al path de Python
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"📁 Proyecto configurado en: {PROJECT_ROOT}")

# Verificar estructura básica
src_path = PROJECT_ROOT / "src"
if src_path.exists():
    print("✅ Estructura del proyecto verificada")
else:
    print("❌ Error: No se encontró la carpeta 'src'. Verifica la instalación.")

---
## 📊 Sección 4: Utilidades de Monitoreo de Recursos

In [ ]:
import torch
import psutil
import time
from datetime import datetime, timedelta
from IPython.display import clear_output

class ResourceMonitor:
    """Monitor de recursos GPU/RAM/Tiempo"""
    
    def __init__(self, max_training_hours=10):
        self.start_time = time.time()
        self.max_training_seconds = max_training_hours * 3600
        self.max_training_hours = max_training_hours
        
    def get_gpu_usage(self):
        """Retorna uso de GPU en %"""
        if not torch.cuda.is_available():
            return 0.0
        try:
            return torch.cuda.utilization()
        except:
            return 0.0
    
    def get_gpu_memory(self):
        """Retorna memoria GPU usada/total en GB"""
        if not torch.cuda.is_available():
            return 0.0, 0.0
        allocated = torch.cuda.memory_allocated() / 1e9  # GB
        reserved = torch.cuda.memory_reserved() / 1e9
        return allocated, reserved
    
    def get_ram_usage(self):
        """Retorna uso de RAM en GB"""
        mem = psutil.virtual_memory()
        return mem.used / 1e9, mem.total / 1e9
    
    def get_elapsed_time(self):
        """Retorna tiempo transcurrido y restante"""
        elapsed = time.time() - self.start_time
        remaining = max(0, self.max_training_seconds - elapsed)
        return elapsed, remaining
    
    def should_stop(self):
        """True si se acerca al límite de tiempo (90%)"""
        elapsed, remaining = self.get_elapsed_time()
        return remaining < (self.max_training_seconds * 0.1)  # 10% restante
    
    def get_status_str(self):
        """Retorna string con estado de recursos"""
        gpu_usage = self.get_gpu_usage()
        gpu_mem_used, gpu_mem_reserved = self.get_gpu_memory()
        ram_used, ram_total = self.get_ram_usage()
        elapsed, remaining = self.get_elapsed_time()
        
        elapsed_str = str(timedelta(seconds=int(elapsed)))
        remaining_str = str(timedelta(seconds=int(remaining)))
        
        status = (
            f"📊 RECURSOS:\n"
            f"  GPU Utilization: {gpu_usage:.1f}%\n"
            f"  GPU Memory: {gpu_mem_used:.2f}GB / {gpu_mem_reserved:.2f}GB\n"
            f"  RAM: {ram_used:.2f}GB / {ram_total:.2f}GB ({ram_used/ram_total*100:.1f}%)\n"
            f"  \n"
            f"⏰ TIEMPO:\n"
            f"  Transcurrido: {elapsed_str}\n"
            f"  Restante: {remaining_str} (de {self.max_training_hours}h máximo)\n"
        )
        return status

print("✅ ResourceMonitor definido")

---
## ⚙️ Sección 5: Configuración del Experimento

In [ ]:
from types import SimpleNamespace
import torch

# ============================================================================
# 🎯 CONFIGURACIÓN DEL CURRICULUM (FASES DE ENTRENAMIENTO)
# ============================================================================

# Definimos una lista de fases. El sistema ejecutará una tras otra.
# Si una fase ya está completada (existe checkpoint final), pasará a la siguiente.

TRAINING_PHASES = [
    # --- FASE 1: ESTABILIDAD DEL VACÍO ---
    {
        "PHASE_NAME": "Fase1_Vacuum_Stability",
        "LOAD_FROM_PHASE": None,  # Empezar desde cero
        
        "MODEL_ARCHITECTURE": "UNET",
        "MODEL_PARAMS": {
            "d_state": 4,           # Dimensión baja para aprender rápido
            "hidden_channels": 32,
        },
        
        "GRID_SIZE_TRAINING": 32,
        "QCA_STEPS_TRAINING": 50,
        "LR_RATE_M": 1e-4,
        "GAMMA_DECAY": 0.001,       # Poco decaimiento
        
        "TOTAL_EPISODES": 500,
        "SAVE_EVERY_EPISODES": 50,
    },
    
    # --- FASE 2: EMERGENCIA DE MATERIA ---
    {
        "PHASE_NAME": "Fase2_Matter_Emergence",
        "LOAD_FROM_PHASE": "Fase1_Vacuum_Stability", # Cargar cerebro de Fase 1
        
        "MODEL_ARCHITECTURE": "UNET",
        "MODEL_PARAMS": {
            "d_state": 4,           # Misma dimensión
            "hidden_channels": 64,  # Más capacidad (neuronas)
        },
        
        "GRID_SIZE_TRAINING": 64,   # Grid más grande
        "QCA_STEPS_TRAINING": 100,
        "LR_RATE_M": 5e-5,          # LR más fino
        "GAMMA_DECAY": 0.01,        # Más presión evolutiva
        
        "TOTAL_EPISODES": 1000,
        "SAVE_EVERY_EPISODES": 20,
    },
]

# Configuración Global
GLOBAL_CONFIG = {
    "DRIVE_SYNC_EVERY": 50,
    "MAX_TRAINING_HOURS": 10,     # Tiempo total para TODAS las fases
    "AUTO_RESUME": True,
    "MAX_CHECKPOINTS_TO_KEEP": 3,
}

# Detectar dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directorios base
EXPERIMENT_ROOT_NAME = "MultiPhase_Experiment_v1" # Nombre carpeta raíz
BASE_CHECKPOINT_DIR = DRIVE_CHECKPOINT_DIR / EXPERIMENT_ROOT_NAME
BASE_LOG_DIR = DRIVE_LOGS_DIR / EXPERIMENT_ROOT_NAME
BASE_CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
BASE_LOG_DIR.mkdir(parents=True, exist_ok=True)

print("=" * 70)
print("📊 CONFIGURACIÓN DEL CURRICULUM")
print("=" * 70)
print(f"Experimento Raíz: {EXPERIMENT_ROOT_NAME}")
print(f"Total Fases: {len(TRAINING_PHASES)}")
print(f"Device: {device}")
print("=" * 70)

for i, phase in enumerate(TRAINING_PHASES):
    print(f"\n🔹 FASE {i+1}: {phase['PHASE_NAME']}")
    print(f"   Load From: {phase['LOAD_FROM_PHASE']}")
    print(f"   Model: {phase['MODEL_ARCHITECTURE']} (d={phase['MODEL_PARAMS']['d_state']}, ch={phase['MODEL_PARAMS']['hidden_channels']})")
    print(f"   Grid: {phase['GRID_SIZE_TRAINING']}x{phase['GRID_SIZE_TRAINING']}")
    print(f"   Episodes: {phase['TOTAL_EPISODES']}")


---
## 🔄 Sección 6: Verificar/Cargar Checkpoint Existente (Auto-Resume)

In [ ]:
import glob
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def find_latest_checkpoint(checkpoint_dir):
    """Encuentra el último checkpoint en un directorio"""
    checkpoints = list(checkpoint_dir.glob("*.pth"))
    if not checkpoints:
        return None
    
    # Buscar last_model.pth primero
    last_model = checkpoint_dir / "last_model.pth"
    if last_model.exists():
        return str(last_model)
    
    # Si no, buscar el más reciente
    latest = max(checkpoints, key=lambda p: p.stat().st_mtime)
    return str(latest)

# Buscar checkpoint existente
checkpoint_path = None
resume_from_episode = 0

if exp_cfg.AUTO_RESUME:
    print("🔍 Buscando checkpoint existente...")
    
    # Primero buscar en Drive
    drive_checkpoint = find_latest_checkpoint(EXPERIMENT_CHECKPOINT_DIR)
    if drive_checkpoint:
        checkpoint_path = drive_checkpoint
        print(f"✅ Checkpoint encontrado en Drive: {Path(checkpoint_path).name}")
    else:
        # Si no hay en Drive, buscar en local
        local_checkpoint = find_latest_checkpoint(LOCAL_CHECKPOINT_DIR)
        if local_checkpoint:
            checkpoint_path = local_checkpoint
            print(f"✅ Checkpoint encontrado localmente: {Path(checkpoint_path).name}")
    
    if checkpoint_path:
        # Cargar checkpoint para ver en qué episodio estamos
        try:
            ckpt_data = torch.load(checkpoint_path, map_location='cpu')
            resume_from_episode = ckpt_data.get('episode', 0)
            print(f"🔄 Continuando desde episodio {resume_from_episode}")
        except Exception as e:
            logger.warning(f"⚠️ Error leyendo checkpoint: {e}")
            checkpoint_path = None

if checkpoint_path is None:
    print("🆕 No se encontró checkpoint previo. Iniciando desde cero.")

# Calcular episodios restantes
episodes_remaining = max(0, exp_cfg.TOTAL_EPISODES - resume_from_episode)
print(f"\n📈 Episodios restantes: {episodes_remaining}/{exp_cfg.TOTAL_EPISODES}")

from src.trainers.qc_trainer_v4 import QC_Trainer_v4
from src.model_loader import instantiate_model, load_weights, load_checkpoint_data
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import json
import shutil

# Inicializar monitor de recursos GLOBAL
monitor = ResourceMonitor(max_training_hours=GLOBAL_CONFIG["MAX_TRAINING_HOURS"])

# Función auxiliar para guardar en Drive
def sync_checkpoint_to_drive(local_path, drive_dir, filename=None):
    """Copia checkpoint local a Drive"""
    try:
        if filename is None:
            filename = Path(local_path).name
        drive_path = drive_dir / filename
        shutil.copy2(local_path, drive_path)
        # logger.info(f"💾 Checkpoint sincronizado a Drive: {filename}")
        return True
    except Exception as e:
        print(f"❌ Error sincronizando a Drive: {e}")
        return False

# ============================================================================
# 🔄 LOOP PRINCIPAL DE FASES
# ============================================================================

for phase_idx, phase_cfg in enumerate(TRAINING_PHASES):
    PHASE_NAME = phase_cfg["PHASE_NAME"]
    
    print("\n" + "#" * 70)
    print(f"🚀 INICIANDO FASE {phase_idx+1}/{len(TRAINING_PHASES)}: {PHASE_NAME}")
    print("#" * 70)
    
    # 1. Configuración de Directorios para esta Fase
    PHASE_CHECKPOINT_DIR = BASE_CHECKPOINT_DIR / PHASE_NAME
    PHASE_LOG_DIR = BASE_LOG_DIR / PHASE_NAME
    PHASE_CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
    PHASE_LOG_DIR.mkdir(parents=True, exist_ok=True)
    
    LOCAL_PHASE_DIR = PROJECT_ROOT / "output" / "checkpoints" / EXPERIMENT_ROOT_NAME / PHASE_NAME
    LOCAL_PHASE_DIR.mkdir(parents=True, exist_ok=True)
    
    # 2. Verificar si la fase ya está completada
    final_marker = PHASE_CHECKPOINT_DIR / "PHASE_COMPLETED.marker"
    if final_marker.exists() and GLOBAL_CONFIG["AUTO_RESUME"]:
        print(f"✅ Fase {PHASE_NAME} ya completada. Saltando...")
        continue

    # 3. Preparar Configuración de Fase
    current_exp_cfg = SimpleNamespace(**phase_cfg)
    current_exp_cfg.MODEL_PARAMS = SimpleNamespace(**phase_cfg["MODEL_PARAMS"])
    current_exp_cfg.DEVICE = device
    
    # 4. Instanciar Modelo
    print(f"🛠️ Instanciando modelo {phase_cfg['MODEL_ARCHITECTURE']}...")
    model = instantiate_model(current_exp_cfg)
    
    # 5. Cargar Pesos (Lógica de Transición)
    resume_from_episode = 0
    weights_loaded = False
    
    # A) Intentar resumir la propia fase (si se interrumpió)
    if GLOBAL_CONFIG["AUTO_RESUME"]:
        latest_ckpt = find_latest_checkpoint(PHASE_CHECKPOINT_DIR)
        if not latest_ckpt:
            latest_ckpt = find_latest_checkpoint(LOCAL_PHASE_DIR)
            
        if latest_ckpt:
            print(f"🔄 Resumiendo fase actual desde: {Path(latest_ckpt).name}")
            ckpt_data = load_checkpoint_data(latest_ckpt)
            if ckpt_data:
                load_weights(model, ckpt_data) # Strict=True porque es el mismo modelo
                resume_from_episode = ckpt_data.get('episode', 0)
                weights_loaded = True
    
    # B) Si no se resume, cargar de fase anterior (Transfer Learning)
    if not weights_loaded and phase_cfg["LOAD_FROM_PHASE"]:
        prev_phase_name = phase_cfg["LOAD_FROM_PHASE"]
        print(f"📥 Buscando pesos de fase anterior: {prev_phase_name}")
        
        prev_dir = BASE_CHECKPOINT_DIR / prev_phase_name
        best_prev = prev_dir / "best_model.pth"
        if not best_prev.exists():
             best_prev = find_latest_checkpoint(prev_dir)
             
        if best_prev and Path(best_prev).exists():
            print(f"✅ Cargando pesos previos de: {Path(best_prev).name}")
            ckpt_data = load_checkpoint_data(str(best_prev))
            
            # CRÍTICO: strict=False para permitir cambio de arquitectura (ej: d_state 4 -> 8)
            missing, unexpected = model.load_state_dict(ckpt_data['model_state_dict'], strict=False)
            print(f"ℹ️ Transfer Learning: {len(missing)} capas nuevas inicializadas, {len(unexpected)} capas descartadas.")
            weights_loaded = True
        else:
            print(f"⚠️ No se encontraron pesos de la fase anterior. Iniciando desde cero (aleatorio).")
            
    if not weights_loaded:
        print("🆕 Iniciando fase con pesos aleatorios.")

    # 6. Inicializar Trainer
    trainer = QC_Trainer_v4(
        experiment_name=f"{EXPERIMENT_ROOT_NAME}/{PHASE_NAME}", # Subdirectorio en logs internos
        model=model,
        device=device,
        lr=phase_cfg["LR_RATE_M"],
        grid_size=phase_cfg["GRID_SIZE_TRAINING"],
        qca_steps=phase_cfg["QCA_STEPS_TRAINING"],
        gamma_decay=phase_cfg["GAMMA_DECAY"],
        max_checkpoints_to_keep=GLOBAL_CONFIG["MAX_CHECKPOINTS_TO_KEEP"]
    )
    
    # Hack para redirigir checkpoints del trainer a nuestra carpeta de fase local
    trainer.checkpoint_dir = str(LOCAL_PHASE_DIR)
    
    # Tracking local
    training_log = {"episodes": [], "losses": []}
    
    # 7. Loop de Episodios
    print(f"\n▶️ Entrenando {phase_cfg['TOTAL_EPISODES'] - resume_from_episode} episodios...")
    
    try:
        for episode in range(resume_from_episode, phase_cfg["TOTAL_EPISODES"]):
            
            # Check tiempo global
            if monitor.should_stop():
                print(f"\n⏰ TIEMPO GLOBAL AGOTADO en Fase {PHASE_NAME}, Ep {episode}")
                trainer.save_checkpoint(episode, is_best=False)
                sync_checkpoint_to_drive(str(LOCAL_PHASE_DIR / f"checkpoint_ep{episode}.pth"), PHASE_CHECKPOINT_DIR)
                raise KeyboardInterrupt("Global Timeout") # Salir de todo
                
            # Entrenar
            epoch_result = trainer.train_episode(episode)
            loss = epoch_result.get("loss_total", 0)
            
            training_log["episodes"].append(episode)
            training_log["losses"].append(loss)
            
            # Guardar/Sync
            if (episode + 1) % phase_cfg["SAVE_EVERY_EPISODES"] == 0:
                is_best = (episode + 1) % (phase_cfg["SAVE_EVERY_EPISODES"] * 2) == 0
                trainer.save_checkpoint(episode, is_best=is_best)
                
            if (episode + 1) % GLOBAL_CONFIG["DRIVE_SYNC_EVERY"] == 0:
                # Sync last & best
                sync_checkpoint_to_drive(str(LOCAL_PHASE_DIR / "last_checkpoint.pth"), PHASE_CHECKPOINT_DIR, "last_model.pth")
                if (LOCAL_PHASE_DIR / "best_model.pth").exists():
                    sync_checkpoint_to_drive(str(LOCAL_PHASE_DIR / "best_model.pth"), PHASE_CHECKPOINT_DIR)
                print(f"☁️ Sync Drive Ep {episode}")

            # Visualización simple
            if (episode + 1) % 10 == 0:
                print(f"Ep {episode+1}/{phase_cfg['TOTAL_EPISODES']} | Loss: {loss:.4f} | {monitor.get_status_str().splitlines()[0]}")
        
        # Fin de Fase
        print(f"\n✅ FASE {PHASE_NAME} COMPLETADA")
        
        # Guardar marcador de completado
        with open(final_marker, 'w') as f:
            f.write(f"Completed at {datetime.now()}")
            
        # Sync final
        sync_checkpoint_to_drive(str(LOCAL_PHASE_DIR / "best_model.pth"), PHASE_CHECKPOINT_DIR)
        
    except KeyboardInterrupt:
        print("\n🛑 Entrenamiento interrumpido.")
        break
    except Exception as e:
        print(f"\n❌ Error en fase {PHASE_NAME}: {e}")
        raise e

print("\n" + "="*70)
print("🏁 ENTRENAMIENTO FINALIZADO")
print("="*70)


In [ ]:
from src.trainers.qc_trainer_v4 import QC_Trainer_v4
from src.model_loader import instantiate_model, load_weights, load_checkpoint_data
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import json

# Inicializar monitor de recursos
monitor = ResourceMonitor(max_training_hours=exp_cfg.MAX_TRAINING_HOURS)

# Instanciar modelo
model = instantiate_model(exp_cfg)

# Cargar pesos si hay checkpoint
if checkpoint_path:
    ckpt_data = load_checkpoint_data(checkpoint_path)
    if ckpt_data:
        load_weights(model, ckpt_data)

# Inicializar trainer
trainer = QC_Trainer_v4(
    experiment_name=exp_cfg.EXPERIMENT_NAME,
    model=model,
    device=device,
    lr=exp_cfg.LR_RATE_M,
    grid_size=exp_cfg.GRID_SIZE_TRAINING,
    qca_steps=exp_cfg.QCA_STEPS_TRAINING,
    gamma_decay=exp_cfg.GAMMA_DECAY,
    max_checkpoints_to_keep=exp_cfg.MAX_CHECKPOINTS_TO_KEEP
)

# Tracking de métricas
training_log = {
    "experiment_name": exp_cfg.EXPERIMENT_NAME,
    "config": EXPERIMENT_CONFIG,
    "episodes": [],
    "losses": [],
    "metrics": [],
    "checkpoints_saved": []
}

# Función auxiliar para guardar en Drive
def sync_checkpoint_to_drive(local_path, episode_num):
    """Copia checkpoint local a Drive"""
    try:
        import shutil
        filename = Path(local_path).name
        drive_path = EXPERIMENT_CHECKPOINT_DIR / filename
        shutil.copy2(local_path, drive_path)
        logger.info(f"💾 Checkpoint sincronizado a Drive: {filename}")
        return True
    except Exception as e:
        logger.error(f"❌ Error sincronizando a Drive: {e}")
        return False

# ENTRENAMIENTO PROGRESIVO
print("\n" + "=" * 70)
print("🎯 INICIANDO ENTRENAMIENTO PROGRESIVO")
print("=" * 70)

try:
    for episode in range(resume_from_episode, exp_cfg.TOTAL_EPISODES):
        
        # Verificar límite de tiempo ANTES de empezar episodio
        if monitor.should_stop():
            print("\n⏰ ¡LÍMITE DE TIEMPO ALCANZADO!")
            print("💾 Guardando checkpoint de emergencia...")
            
            # Guardar checkpoint de emergencia
            trainer.save_checkpoint(episode, is_best=False)
            
            # Sincronizar a Drive
            last_checkpoint = LOCAL_CHECKPOINT_DIR / "last_model.pth"
            if last_checkpoint.exists():
                sync_checkpoint_to_drive(str(last_checkpoint), episode)
            
            print(f"✅ Entrenamiento detenido en episodio {episode}")
            print(f"📊 Progreso: {episode}/{exp_cfg.TOTAL_EPISODES} episodios ({episode/exp_cfg.TOTAL_EPISODES*100:.1f}%)")
            break
        
        # Entrenar episodio
        epoch_result = trainer.train_episode(episode)
        
        # Registrar métricas
        training_log["episodes"].append(episode)
        training_log["losses"].append(epoch_result.get("loss_total", 0))
        training_log["metrics"].append(epoch_result)
        
        # Guardar checkpoint periódicamente
        if (episode + 1) % exp_cfg.SAVE_EVERY_EPISODES == 0:
            is_best = (episode + 1) % (exp_cfg.SAVE_EVERY_EPISODES * 5) == 0  # Cada 50 eps = best candidate
            trainer.save_checkpoint(episode, is_best=is_best)
            training_log["checkpoints_saved"].append(episode)
        
        # Sincronizar a Drive periódicamente
        if (episode + 1) % exp_cfg.DRIVE_SYNC_EVERY == 0:
            print(f"\n💾 Sincronizando checkpoint a Drive (episodio {episode})...")
            last_checkpoint = LOCAL_CHECKPOINT_DIR / "last_model.pth"
            if last_checkpoint.exists():
                sync_checkpoint_to_drive(str(last_checkpoint), episode)
            
            # También sincronizar mejores modelos
            best_checkpoint = LOCAL_CHECKPOINT_DIR / "best_model.pth"
            if best_checkpoint.exists():
                sync_checkpoint_to_drive(str(best_checkpoint), episode)
        
        # Visualización cada 10 episodios
        if (episode + 1) % 10 == 0:
            clear_output(wait=True)
            
            print("=" * 70)
            print(f"📈 PROGRESO: Episodio {episode + 1}/{exp_cfg.TOTAL_EPISODES} ({(episode+1)/exp_cfg.TOTAL_EPISODES*100:.1f}%)")
            print("=" * 70)
            
            # Mostrar recursos
            print(monitor.get_status_str())
            
            # Mostrar últimas métricas
            if len(training_log["losses"]) > 0:
                recent_losses = training_log["losses"][-10:]
                avg_loss = sum(recent_losses) / len(recent_losses)
                print(f"\n📊 MÉTRICAS (últimos 10 episodios):")
                print(f"  Loss promedio: {avg_loss:.4f}")
                print(f"  Loss actual: {training_log['losses'][-1]:.4f}")
            
            # Gráfico de pérdida
            if len(training_log["episodes"]) > 1:
                plt.figure(figsize=(10, 4))
                plt.plot(training_log["episodes"], training_log["losses"], 'b-', alpha=0.6)
                plt.xlabel('Episodio')
                plt.ylabel('Loss')
                plt.title(f'{exp_cfg.EXPERIMENT_NAME} - Progreso de Entrenamiento')
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()
    
    print("\n" + "=" * 70)
    print("✅ ENTRENAMIENTO COMPLETADO EXITOSAMENTE")
    print("=" * 70)
    
except KeyboardInterrupt:
    print("\n⚠️ Entrenamiento interrumpido por usuario")
    print("💾 Guardando checkpoint...")
    trainer.save_checkpoint(episode, is_best=False)
    
except Exception as e:
    print(f"\n❌ ERROR DURANTE ENTRENAMIENTO: {e}")
    logger.error(f"Error: {e}", exc_info=True)
    print("💾 Guardando checkpoint de emergencia...")
    try:
        trainer.save_checkpoint(episode, is_best=False)
    except:
        pass
    raise

finally:
    # Guardar log de entrenamiento en Drive
    log_file = EXPERIMENT_LOG_DIR / f"training_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(log_file, 'w') as f:
        json.dump(training_log, f, indent=2)
    print(f"\n📄 Log guardado en: {log_file}")

---
## 📊 Sección 8: Visualización de Resultados Finales

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

if len(training_log["episodes"]) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Pérdida total
    axes[0, 0].plot(training_log["episodes"], training_log["losses"], 'b-', alpha=0.6, label='Loss')
    axes[0, 0].set_xlabel('Episodio')
    axes[0, 0].set_ylabel('Loss Total')
    axes[0, 0].set_title('Evolución de la Pérdida')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].legend()
    
    # 2. Métricas individuales (si están disponibles)
    if len(training_log["metrics"]) > 0 and "survival_rate" in training_log["metrics"][0]:
        survival_rates = [m.get("survival_rate", 0) for m in training_log["metrics"]]
        axes[0, 1].plot(training_log["episodes"], survival_rates, 'g-', alpha=0.6, label='Survival Rate')
        axes[0, 1].set_xlabel('Episodio')
        axes[0, 1].set_ylabel('Survival Rate')
        axes[0, 1].set_title('Tasa de Supervivencia')
        axes[0, 1].grid(True, alpha=0.3)
        axes[0, 1].legend()
    
    # 3. Checkpoints guardados
    axes[1, 0].scatter(training_log["checkpoints_saved"], 
                      [1] * len(training_log["checkpoints_saved"]), 
                      c='red', marker='|', s=100, label='Checkpoint')
    axes[1, 0].set_xlabel('Episodio')
    axes[1, 0].set_yticks([])
    axes[1, 0].set_title('Checkpoints Guardados')
    axes[1, 0].grid(True, alpha=0.3, axis='x')
    axes[1, 0].legend()
    
    # 4. Histograma de pérdidas
    axes[1, 1].hist(training_log["losses"], bins=30, alpha=0.7, color='purple', edgecolor='black')
    axes[1, 1].set_xlabel('Loss')
    axes[1, 1].set_ylabel('Frecuencia')
    axes[1, 1].set_title('Distribución de Pérdidas')
    axes[1, 1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(EXPERIMENT_LOG_DIR / 'training_summary.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\n📊 Gráficos guardados en: {EXPERIMENT_LOG_DIR / 'training_summary.png'}")
else:
    print("⚠️ No hay datos de entrenamiento para visualizar")

---
## 📦 Sección 9: Exportación y Finalización

In [ ]:
from src.model_loader import load_model
import shutil

print("📤 Exportando modelo final...\n")

# Encontrar mejor checkpoint
best_checkpoint = LOCAL_CHECKPOINT_DIR / "best_model.pth"
if not best_checkpoint.exists():
    best_checkpoint = find_latest_checkpoint(LOCAL_CHECKPOINT_DIR)

if best_checkpoint:
    print(f"📥 Cargando modelo desde: {Path(best_checkpoint).name}")
    
    # Cargar modelo
    model = load_model(exp_cfg, str(best_checkpoint))
    model.eval()
    model.to(device)
    
    # Exportar a TorchScript
    example_input = torch.randn(1, 2 * exp_cfg.MODEL_PARAMS.d_state, 
                                exp_cfg.GRID_SIZE_TRAINING, 
                                exp_cfg.GRID_SIZE_TRAINING, device=device)
    
    torchscript_path = DRIVE_EXPORTS_DIR / f"{exp_cfg.EXPERIMENT_NAME}_model.pt"
    
    try:
        traced_model = torch.jit.trace(model, example_input, strict=False)
        traced_model.save(str(torchscript_path))
        print(f"✅ Modelo TorchScript exportado: {torchscript_path.name}")
    except Exception as e:
        print(f"⚠️ Error exportando TorchScript: {e}")
    
    # Copiar mejor checkpoint a Drive
    drive_best_checkpoint = EXPERIMENT_CHECKPOINT_DIR / "best_model_FINAL.pth"
    shutil.copy2(best_checkpoint, drive_best_checkpoint)
    print(f"✅ Mejor checkpoint copiado: {drive_best_checkpoint.name}")
    
    # Generar reporte de entrenamiento
    report_path = EXPERIMENT_LOG_DIR / f"{exp_cfg.EXPERIMENT_NAME}_REPORT.md"
    
    with open(report_path, 'w') as f:
        f.write(f"# Reporte de Entrenamiento: {exp_cfg.EXPERIMENT_NAME}\n\n")
        f.write(f"**Fecha:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write(f"## Configuración\n\n")
        f.write(f"```json\n{json.dumps(EXPERIMENT_CONFIG, indent=2)}\n```\n\n")
        f.write(f"## Resultados\n\n")
        f.write(f"- **Total de episodios completados:** {len(training_log['episodes'])}\n")
        f.write(f"- **Checkpoints guardados:** {len(training_log['checkpoints_saved'])}\n")
        
        if len(training_log["losses"]) > 0:
            f.write(f"- **Loss final:** {training_log['losses'][-1]:.4f}\n")
            f.write(f"- **Loss promedio:** {np.mean(training_log['losses']):.4f}\n")
            f.write(f"- **Loss mínimo:** {min(training_log['losses']):.4f}\n")
        
        f.write(f"\n## Archivos\n\n")
        f.write(f"- Mejor modelo: `{drive_best_checkpoint.name}`\n")
        f.write(f"- TorchScript: `{torchscript_path.name}`\n")
        f.write(f"- Logs: `{EXPERIMENT_LOG_DIR.name}`\n")
    
    print(f"✅ Reporte generado: {report_path.name}")
    
else:
    print("⚠️ No se encontró checkpoint para exportar")

print("\n" + "=" * 70)
print("🎉 EXPORTACIÓN COMPLETADA")
print("=" * 70)
print(f"\n📁 Todos los archivos están en Drive:")
print(f"  - Checkpoints: {EXPERIMENT_CHECKPOINT_DIR}")
print(f"  - Logs: {EXPERIMENT_LOG_DIR}")
print(f"  - Exports: {DRIVE_EXPORTS_DIR}")
print("\n✅ Puedes cerrar el notebook. Todo está guardado en Drive.")

---
## 📝 Notas Finales

### ✅ Checklist de Verificación

- ✓ Checkpoints guardados en Drive cada 50 episodios
- ✓ Logs de entrenamiento persistidos
- ✓ Modelo final exportado a TorchScript
- ✓ Reporte de entrenamiento generado
- ✓ Auto-recuperación configurada para próxima sesión

### 🔄 Para Continuar Entrenamiento

1. Ejecutar este notebook de nuevo
2. Mantener `AUTO_RESUME = True`
3. Ajustar `TOTAL_EPISODES` si quieres más episodios
4. El notebook detectará automáticamente el último checkpoint en Drive

### 📊 Mejores Prácticas

- **Colab Free**: Entrenar en sesiones de 6-8 horas
- **Colab Pro**: Sesiones de 12-20 horas
- **Kaggle**: Aprovechar las 30 horas semanales
- Siempre verificar que Drive esté sincronizando correctamente
- Revisar gráficos cada 50-100 episodios para detectar problemas

---

**¡Feliz entrenamiento! 🚀**